$$\text{Trabajo Final de Comunicaciones Digitales - Parte N° 1}$$

$\text{Diseño del Codificador - Decodificador }$

# Marco Teórico

## Codificador

El **codificador** es el dispositivo que toma el mensaje y produce la palabra código. El mensaje consiste en una secuencia de bits de información que debe ser transformada para su transmisión eficiente.

Si queremos comunicar una secuencia de $k$ bits, existen $2^k$ secuencias distintas y cada una debería ser mapeada a una secuencia de símbolos distinta. Esta relación se expresa matemáticamente como:

$$2^k \leq m^n$$

donde $m$ es el tamaño del alfabeto de símbolos y $n$ es la longitud de la secuencia de símbolos.

### Codificación Binaria Directa

La base matemática para lo que llamamos **codificación binaria directa** consiste en agrupar $k$ bits y interpretarlos como un entero entre $0$ y $2^{k}-1$, el cual se mapea a un símbolo dentro de un alfabeto de tamaño $m = 2^k$.

La fórmula que describe esta transformación es:

$$\text{Símbolo} = \sum_{i=0}^{SF-1} b_i \cdot 2^i$$

donde:
- $b_i$ es el $i$-ésimo bit del grupo
- $SF$ es el Spreading Factor (Factor de Expansión)

## Decodificador

El **decodificador** es el dispositivo responsable de inferir el mensaje original a partir de la señal recibida. En el contexto del canal AWGN (Ruido Blanco Gaussiano Aditivo), el decodificador implementa una estrategia de **Máxima Verosimilitud** (Maximum Likelihood).

### Decodificador de Máxima Verosimilitud

Para un decodificador ML en el canal AWGN en tiempo discreto, se elige una de las secuencias de salida $\mathbf{x}$ que maximiza la función de verosimilitud. La métrica de decisión se expresa como:

$$\langle \mathbf{c}, \mathbf{y} \rangle - \frac{|\mathbf{c}|^2}{2}$$

donde:
- $\mathbf{c}$ es la palabra código candidata
- $\mathbf{y}$ es la señal recibida
- $\langle \mathbf{c}, \mathbf{y} \rangle$ es el producto interno entre ambos vectores
- $|\mathbf{c}|^2$ es la energía de la palabra código

$\text{Desarrollo}$

$$\text{Modulacion de chirp por desplazamiento de frecuencia}$$

En el paper, el proceso de codificación en la modulación por desplazamiento de frecuencia mediante chirps se describe matemáticamente de manera precisa y puede descomponerse en tres etapas fundamentales:

## Etapa 1: Mapeo de bits a símbolo decimal

El proceso comienza con un vector $w(nT_s)$ compuesto por una cantidad determinada de bits. Esta cantidad se denomina **Spreading Factor** (SF). El Spreading Factor es el número de bits por símbolo y se calcula como:

$$SF = \log_2(M)$$

donde $M$ es la cardinalidad de la modulación (la cantidad de símbolos distintos). Por ejemplo, si $SF = 5$, hay $2^5 = 32$ símbolos posibles.

Una vez obtenido el vector $w(nT_s)$, se utiliza la fórmula planteada en el paper para sumar los valores ponderados de los bits del vector $w(nT_s)$ y así generar el valor decimal entero $s(nT_s)$. Este número se utiliza para decidir con qué frecuencia inicial el chirp inicia la transmisión.

### Parámetros de transmisión

Supongamos que el ancho de banda del canal que usamos para la transmisión es $B$ y que transmitimos una muestra cada $T$, con:

$$T = \frac{1}{B}$$

Un símbolo $s(nT_s)$ es enviado a la entrada del modulador cada:

$$T_s = 2^{SF} \cdot T$$

El símbolo $s(nT_s)$ es un número real que se forma usando un vector $w(nT_s)$ de dígitos binarios SF, con SF como un parámetro entero llamado **Factor de Expansión**, el cual normalmente toma valores en $\{7, 8, 9, 10, 11, 12\}$.

El SF (Spreading Factor) es el número de bits que se agrupan en cada símbolo. Cada número representa un entero entre $0$ y $2^{SF} - 1$ y representa cuánto se "esparce" la señal en el tiempo, determinando la cantidad de símbolos que se pueden transmitir.



## Ecuación 1: Codificación de bits a símbolo

El proceso de codificación transforma una secuencia de bits en símbolos que pueden ser transmitidos a través del canal. 

La codificación binaria directa agrupa $k$ bits consecutivos y los interpreta como un número entero entre $0$ y $2^k - 1$. Este número entero se mapea directamente a un símbolo dentro de un alfabeto de tamaño $m = 2^k$. 

La fórmula matemática que describe esta transformación es:

$$s(nT_s) = \sum_{h=0}^{SF-1} w_h(nT_s) \cdot 2^h$$

donde:
- $s(nT_s) \in {0,...,2^SF - 1}$ es el simbolo codificado 
- $w_h(nT_s) \in {0,1}$ son los bits del bloque actual, ordenados desde el menos significativo (LSB).
- $SF$ es el Spreading Factor (Factor de Expansión), que define la cantidad de bits por símbolo.
- $h$ es el índice del bit dentro del bloque

Entonces: 
    Esta operación convierte secuenci sbinarias en números enteros para su modulacíon, permitiendo la reperesentacíon 
    eficiente de la informacíon.
    

## Ecuación 2 y 3: Modulación Chirp 

**Ecuación 2 (forma general):**

$$c(nT_s + kT) = \frac{1}{\sqrt{2^{SF}}} \cdot e^{j2\pi[(s(nT_s)+k) \bmod 2^{SF}]\frac{kT}{B}} \quad (2)$$

donde:
- $c(nT_s + kT)$ es la señal chirp modulada en el tiempo $nT_s + kT$
- $\frac{1}{\sqrt{2^{SF}}}$ es el factor de normalización de energía (asegura energía unitaria para cada símbolo)
- $s(nT_s)$ es el símbolo codificado (de la Ecuación 1)
- $k$ es el índice de muestra dentro del símbolo ($k = 0, 1, ..., 2^{SF}-1$)
- $T$ es el período de muestreo
- $B$ es el ancho de banda del canal
- $\bmod 2^{SF}$ indica la operación módulo $2^{SF}$

### Características principales:

1. **Chirp complejo**: La señal tiene frecuencia linealmente creciente dentro de cada símbolo
2. **Frecuencia instantánea**: Depende de $(s(nT_s)+k) \bmod 2^{SF}$, que representa un corrimiento de frecuencia específico para cada símbolo
3. **Ortogonalidad**: Los diferentes símbolos generan chirps ortogonales entre sí, facilitando la decodificación
4. **Normalización**: El factor $\frac{1}{\sqrt{2^{SF}}}$ mantiene la energía constante por símbolo


  
**Ecuación 3 (forma simplifca:**)


Dado que $T = \frac{1}{B}$, se reemplaza por $T \cdot B = 1$, simplificando la ecuación:

$$c(nT_s + kT) = \frac{1}{\sqrt{2^{SF}}} \cdot e^{j2\pi \cdot \frac{[(s(nT_s)+k) \bmod 2^{SF}] \cdot k}{2^{SF}}} \quad (3)$$

**Pasos** 

1. Definiciíon del tiempo discreto:  

Teniendo en cuento lo definido más arriba: 
 
$$t = nT_s + kT = n \cdot 2^ SF \cdot T + kT$$

2. Codificar la frecuencia en el chirp: 

La modulacíon LoRa usa un chirp de frecuencia creciente, donde la frecuencia instantánea varía linealmente con k, y 
se desplaza según el valor del símbolo $$s(nT_s)$$ 
$$\frac{[(s+k) \bmod 2^{SF}] \cdot k}{2^{SF}}$$


3. Fase acumulada y señal compleja 

$$e^{j2\pi \cdot \frac{[(s+k) \bmod 2^{SF}] \cdot k}{2^{SF}}}$$

Representa una señal compleja cuya fase cambia con el tiempo (índice k).Esa fase varía en funcíon del símbolo transmitido s y de la muestra actual k. 
Como resultado, se genera una onda chirp (señal cuya frecuencia aumenta linealmente con k ). Esto se logra porque la fase del exponente crece de forma cuadrática con k, algo característico de un chirp lineal. 
El desplazamiento s hace que cada símbolo comience su chirp en una frecuencia distinta, lo que permite codificar la información.

4. Normalizacíon de energía 

El factor $\frac{1}{\sqrt{2^{SF}}}$ normaliza la señal para que tenga energía unitaria. 

$\text{Implementación en python}$

<text>La función bits_to_symbols: Agrupa un arreglo plano de bits en bloques de longitud igual al Spreading Factor (SF) y 

convierte cada bloque en un número decimal. Este paso simula lo que en el paper se describe como la generación del símbolo
 
𝑠(𝑛𝑇𝑠) a partir del vector de bits 𝑤(𝑛𝑇𝑠)

In [ ]:
import random
import numpy as np

def bits_to_symbols(bit_array, SF):
    """Agrupa los bits en bloques de SF y los convierte en símbolos enteros."""
    bit_array = np.array(bit_array).reshape(-1, SF)
    powers = 2 ** np.arange(SF)[::-1]
    symbols = bit_array.dot(powers)
    return symbols

<text>La funcion symbols to bits: Invierte el proceso anterior: toma una lista de símbolos y los convierte en sus 
correspondientes representaciones binarias de SF bits. Esto emula la decodificación, recuperando el vector 𝑤(𝑛𝑇𝑠) desde el número 𝑠(𝑛𝑇𝑠).

In [ ]:
def symbols_to_bits(symbols, SF):
    """Convierte los símbolos de vuelta a su representación binaria."""
    bits = ((symbols[:, None] & (1 << np.arange(SF)[::-1])) > 0).astype(int)
    return bits.reshape(-1)

<text>  La funcion simulate_encoder_decoder: Es el núcleo de la simulación. 

1~ Genera bits aleatorios con una distribución uniforme entre 0 y 1 usando np.random.randint.

2~ Codifica: convierte esos bits en símbolos utilizando bits_to_symbols.

3~ Simula la transmisión: en este primer modelo, la transmisión es perfecta. Es decir, los símbolos recibidos son iguales a los transmitidos (sin errores de canal, por ahora).

4~ Decodifica: recupera los bits originales desde los símbolos usando symbols_to_bits.

5~ Calcula el BER: compara los bits transmitidos vs. los recibidos para calcular la tasa de error de bit (Bit Error Rate).

6~ Imprime resultados: muestra los primeros 64 bits transmitidos y decodificados, junto con el BER final y estadísticas generales.

In [ ]:
def simulate_encoder_decoder(SF, total_bits):
    """Genera bits aleatorios, codifica, decodifica y calcula el BER."""
    assert total_bits % SF == 0, "El número total de bits debe ser múltiplo de SF"

    # Generar bits aleatorios con distribución uniforme
    tx_bits = np.random.randint(0, 2, total_bits)

    # Codificación
    tx_symbols = bits_to_symbols(tx_bits, SF)

    # Transmisión simulada perfecta (sin ruido)
    rx_symbols = tx_symbols.copy()  # En canal real, se podría agregar ruido o errores

    # Decodificación
    rx_bits = symbols_to_bits(rx_symbols, SF)

    # Cálculo de BER
    bit_errors = np.sum(tx_bits != rx_bits)
    ber = bit_errors / total_bits

    # Resultados
    print("Bits transmitidos (primeros 64):  ", tx_bits[:64])
    print("Bits decodificados (primeros 64):", rx_bits[:64])
    print(f"\nTotal de bits transmitidos: {total_bits}")
    print(f"Errores totales: {bit_errors}")
    print(f"BER (Bit Error Rate): {ber:.6f}")

# Parámetros de simulación
SF = 7  # Spreading Factor
num_bits = SF * 1000  # Enviar 1000 símbolos

simulate_encoder_decoder(SF, num_bits)